# An Example of using WonderD149Data package to extract data correlation from Natality, 2016-2021 expanded database by CDC

First adding the package to sys path.

In [1]:
import sys, itertools
from pathlib import Path
from bokeh.io import output_notebook, show, reset_output
sys.path.append('../src')

Now importing helper from the package to investigate all required options

In [2]:
import wonderD149Data.data.helper as hp
import wonderD149Data.wonderD149Data as wd

Reloading the custom modules explicitly to incorporate live changes in src code

In [3]:
import importlib

importlib.reload(hp)
importlib.reload(wd)

<module 'wonderD149Data.wonderD149Data' from '/Users/abhijithdasharathi/Study/UCSD/Winter_23/ECE143/CDC_Project/wonderD149Data/examples/../src/wonderD149Data/wonderD149Data.py'>

creating wonderD149Data Object and getting data

In [4]:
# run below line for updating the cache manually (it usually takes 20 min to manually create data)
# col_data_cleaned = hp.getBirthAnalysisData(False)
col_data_cleaned = hp.getBirthAnalysisData()


looking for cached values
cached data found


Selecting columns of high negative correlations

In [5]:

# Create correlation matrix
corr_matrix = col_data_cleaned.corr()

# Find features with correlation greater than 0.995
to_investigate = [(row,column) for row,column in itertools.product(corr_matrix.index,corr_matrix.columns) if corr_matrix[column][row] < -0.75 and hp.is_Different_Category(row,column)]
print(len(to_investigate),'combinations')


38 combinations


Selecting columns of high positive correlations

In [6]:
# Find features with correlation lesser than 0.01
to_investigate_low = [(row,column) for row,column in itertools.product(corr_matrix.index,corr_matrix.columns) if corr_matrix[column][row] > 0.99999999 and hp.is_Different_Category(row,column)]
print(len(to_investigate_low),'combinations')

34 combinations


Creating graph for high negative correlation

In [7]:
Gn = hp.createCorrelationGraph(corr_matrix,to_investigate)

Creating graph for high positive correlation

In [8]:
Gp = hp.createCorrelationGraph(corr_matrix,to_investigate_low)

Visualize the generated graphs

In [9]:
#Choose a title!
title = 'Pregnancy_related_births_data_high_negative_correlation'

path = Path('/Users/abhijithdasharathi/Study/UCSD/Winter_23/ECE143/CDC_Project/wonderD149Data/examples')

plot = hp.plotNetworkGraph(Gn,title,path)
try:
    reset_output()
    output_notebook()
    show(plot)
except:
    output_notebook()
    show(plot)

/Users/abhijithdasharathi/Study/UCSD/Winter_23/ECE143/CDC_Project/wonderD149Data/examples/../src/wonderD149Data/data/helper.py:357: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(plot, filename=f"{path}/{title}.html")
/Users/abhijithdasharathi/Study/UCSD/Winter_23/ECE143/CDC_Project/wonderD149Data/examples/../src/wonderD149Data/data/helper.py:357: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(plot, filename=f"{path}/{title}.html")


Loading BokehJS ...

Loading BokehJS ...

In [10]:
#Choose a title!
title = 'Pregnancy_related_births_data_high_positive_correlation'

path = Path('/Users/abhijithdasharathi/Study/UCSD/Winter_23/ECE143/CDC_Project/wonderD149Data/examples')

plot = hp.plotNetworkGraph(Gp,title,path)
try:
    reset_output()
    output_notebook()
    show(plot)
except:
    output_notebook()
    show(plot)

Loading BokehJS ...

Loading BokehJS ...